In [3]:
from importlib import reload
import load_data
reload(load_data)
from load_data import *


import numpy as np
from langdetect import detect
import translitcodec
import codecs

# 1. Standardowa analiza

In [5]:
path_to_file = "data/title.akas.tsv.gz"
df1 = load_to_dataframe(path_to_file)
df1 = df1.drop(columns=["language", "types", "attributes"])

path_to_file = "data/title.basics.tsv.gz"
df3 = load_to_dataframe(path_to_file)
df3 = df3.loc[:, ["tconst", "titleType"]]

path_to_file = "data/title.ratings.tsv.gz"
df7 = load_to_dataframe(path_to_file)

df = df1.merge(df3,
              how="left",
              left_on="titleId",
              right_on="tconst"
)

# wybieramy tylko "movie" i niektóre kolumny
df = df.loc[df["titleType"] == "movie", ["titleId", "title", "region", "isOriginalTitle"]]

# zamieniamy wszystkie tytuły na lowercase i dekodujemy narodowe znaki
df["title"] = df["title"].astype(str).str.lower().apply(lambda x: codecs.encode(str(x), 'translit/short'))

# wydostajemy kandydatów na filmy z których odzyskamy kraje
# zgodnie z uwagą w PDFie od prowadzących
df_think = df.loc[df["isOriginalTitle"] == 0].merge(
    df.loc[df["isOriginalTitle"] == 1],
    on="titleId",
    how="left",
    suffixes=("", "_y")
).query(
    "title == title_y"
).drop(
    labels=["title_y", "region_y", "isOriginalTitle", "isOriginalTitle_y"],
    axis=1
)

# grupujemy po ID tytułu
# potem zliczamy ile było przypisanych regionów
# tam, gdzie był jeden, daje się zidentyfikować jednoznacznie
disambig = df_think.loc[:, ["titleId", "region"]].groupby(
    by=["titleId"]
).count()["region"] == 1

df_with_known_countries = df_think.loc[df_think["titleId"].isin(disambig.index[disambig == True])]

df_analysis = df_with_known_countries.merge(df7,
                                            how="left",
                                            left_on="titleId",
                                            right_on="tconst"
).drop(
    columns=["tconst"]
).dropna()

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


# 2. Model językowy

In [6]:
# wspieramy rozpoznawanie języka przez bibliotekę 'langdetect'
def mydetect(x):
    try:
        return detect(x).upper()
    except:
        return pd.NA

# funkcje do .apply()
def myfunc_EC(x):
    if x[5] != 'ES':
        x[2] = x[5]
    return x

def myfunc_TR(x):
    if x[5] != 'TR':
        x[2] = x[5]
    return x
    
# usuwamy dziwne filmy z Ekwadoru
df_EC = df_analysis.loc[df_analysis["region"] == 'EC'].copy()
df_EC.loc[:, "detected"] = df_EC["title"].apply(mydetect)

false_EC = df_EC.loc[df_EC["detected"] != 'ES']["titleId"]
df_EC = df_EC.apply(myfunc_EC, axis=1).drop(columns=["detected"])

df_analysis.loc[df_EC.index] = df_EC

In [8]:
# # usuwamy dziwne filmy z Turcji
# df_TR = df_analysis.loc[df_analysis["region"] == 'TR'].copy()
# df_TR.loc[:, "detected"] = df_TR["title"].apply(mydetect)
# df_TR = df_TR.dropna()

# false_TR = df_TR.loc[df_TR["detected"] != 'TR']["titleId"]
# df_TR = df_TR.apply(myfunc_TR, axis=1).drop(columns=["detected"])

# df_analysis.loc[df_TR.index] = df_TR

# 3. Zasadnicza część zadania

In [ ]:
K = 10000 # próg odcięcia
N = 1000 # ile pierwszych filmów bierzemy
df_analysis.loc[df_analysis["numVotes"] > K].sort_values(
    by="averageRating", ascending=False
).head(N).loc[:, ["titleId", "region"]].groupby(by="region").count()

In [43]:
from importlib import reload
import task1
reload(task1)
from task1 import *

In [22]:
df, df3 = load_dataframes(path1="data/title.akas.tsv.gz",
                                path2="data/title.basics.tsv.gz",
                                path3="data/title.ratings.tsv.gz")

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


In [27]:
df_analysis = prepareDataset(df, df3)

In [28]:
df_analysis.head(10)

,titleId,title,region,averageRating,numVotes
0,tt0000147,the corbett-fitzsimmons fight,US,5.2,517.0
1,tt0000502,bohemios,ES,4.4,17.0
2,tt0000591,l'enfant prodigue,FR,5.7,25.0
3,tt0000615,robbery under arms,AU,4.3,25.0
4,tt0000630,amleto,IT,2.9,27.0
5,tt0000675,don quijote,ES,4.2,20.0
11,tt0000862,faldgruben,DK,4.4,17.0
16,tt0000941,locura de amor,ES,4.5,29.0
21,tt0001010,proteccion de un convoy de viveres en el puent...,ES,4.2,15.0
22,tt0001028,salome mad,GB,4.0,20.0


In [44]:
properAnalysis(df_analysis, K = 1000, N = 100)

Top 100 countries - ranking
1. India
2. Turkey
3. Romania
4. Hungary
5. Bangladesh
6. Croatia
7. XYU
8. SW
9. Latvia
10. YUCS
11. Finland
12. Japan
13. Georgia
14. Canada
15. Poland
16. France
17. Slovakia
18. SQ
19. Timor-Leste
20. Germany
21. Ukraine
22. United States
23. China
24. Mexico
